### Initialization

In [1]:
import glob
import re
import textdistance
import numpy
import pandas
import editdistance
import udapi

In [2]:
from utils.text_extraction import *
from utils.document_processing import *
from utils.subject_extraction import *
from utils.similarity import *
from utils.context_extraction import *

In [3]:
import psycopg2
psycopg2_conn = psycopg2.connect(dbname='public_contracts', user='postgres', password='admin', host='localhost', port='5432')

In [4]:
from ufal.udpipe import *
# load model from the given file;
# if the file does not exist, expect a Segmentation fault
udpipe_model = Model.load("../model/udpipe/udpipe-ud-2.5-191206/czech-pdt-ud-2.5-191206.udpipe")

In [5]:
# create a UDPipe processing pipeline with the loaded model,
# with "horizontal" input (a sentence with space-separated tokens),
# default setting for tagger and parser,
# and CoNLL-U output
udp_pipeline = Pipeline(udpipe_model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")

### Documents 

In [6]:
from utils.document_processing import *

In [7]:
loader = DatabaseContractsLoader(psycopg2_conn)
loader.load_documents()
loader.prepare_documents()
loader.prepare_contracts()
df_contracts = pandas.DataFrame.from_dict(loader._contracts, orient='index').reset_index()

Running query: select * from document where processed=True
Preparing total 1008 documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 60.0%
Progress: 70.0%
Progress: 80.0%
Progress: 90.0%
Progress: 100.0%
Preparing total 159 contracts
Progress: 0.0%
Progress: 10.0%
Progress: 19.0%
Progress: 29.0%
Progress: 38.0%
Progress: 48.0%
Progress: 57.0%
Progress: 67.0%
Progress: 76.0%
Progress: 85.0%
Progress: 95.0%


### Subject context

In [8]:
from utils.context_extraction import *

In [9]:
%%time

extractor = AdvancedSubjectContextExtractor()
df_contracts['subj_context'] = df_contracts['text'].apply(lambda text: extractor.process(text))

Wall time: 55.2 s


In [10]:
df_contracts['merged_context'] = df_contracts['subj_context'].apply(lambda contexts: '====='.join(contexts))

In [11]:
df_contracts

,index,docs,text,subj_context,merged_context
0,2,"[{'id': 1, 'text': ' SMLOUVA O PROVÁDĚNÍ...",\n<FILE id=1>\n\n \n\n \n\nSMLOUVA O PROVÁDĚNÍ...,[Předmět smlouvy a podmínky provádění prací v ...,Předmět smlouvy a podmínky provádění prací v L...
1,3,"[{'id': 3, 'text': ' SMLOUVA © DÍLO č.j....",\n<FILE id=3>\n\n \n\n \n\nSMLOUVA © DÍLO\nč.j...,[Předmět smlouvy a předmět díla\n\nPředmětem t...,Předmět smlouvy a předmět díla\n\nPředmětem té...
2,6,"[{'id': 26, 'text': ' SMLOUVA O PROVÁDĚN...",\n<FILE id=26>\n\n \n\n \n\nSMLOUVA O PROVÁDĚN...,[Předmět smlouvy a podmínky provádění prací v ...,Předmět smlouvy a podmínky provádění prací v L...
3,10,"[{'id': 35, 'text': 'N Á V R H SMLOUVA O DÍ...",\n<FILE id=35>\nN Á V R H\n\nSMLOUVA O DÍLO\...,[Předmět díla\n1. \nPředmětem smlouvy je reno...,Předmět díla\n1. \nPředmětem smlouvy je renov...
4,7,"[{'id': 27, 'text': ' SMLOUVA © DÍLO č.j...",\n<FILE id=27>\n\n \n\n \n\nSMLOUVA © DÍLO\nč....,[Předmět smlouvy a předmět díla\n\nPředmětem t...,Předmět smlouvy a předmět díla\n\nPředmětem té...
...,...,...,...,...,...
154,839,"[{'id': 4117, 'text': 'EVROPSKÁ UNIE 6 9 EVROP...",\n<FILE id=4117>\nEVROPSKÁ UNIE 6 9\nEVROPSKÝ ...,[Předmět smlouvy\n\nPředmětem této Smlouvy jso...,Předmět smlouvy\n\nPředmětem této Smlouvy jsou...
155,840,"[{'id': 4120, 'text': 'c o X bi EVROPSKÁ UNIE ...",\n<FILE id=4120>\nc o X\nbi EVROPSKÁ UNIE . 20...,[Předmět veřejné zakázky\n\nPředmětem veřejné ...,Předmět veřejné zakázky\n\nPředmětem veřejné z...
156,841,"[{'id': 4121, 'text': '. a EVROPSKÁ UNIE . G0...",\n<FILE id=4121>\n.\n\na EVROPSKÁ UNIE .\nG0. ...,[Předmět smlouvy\n\nPředmětem této Smlouvy jso...,Předmět smlouvy\n\nPředmětem této Smlouvy jsou...
157,842,"[{'id': 4124, 'text': 'EVROPSKÁ UNIE p »007-1...",\n<FILE id=4124>\nEVROPSKÁ UNIE p\n\n»007-12\n...,[Předmět smlouvy\n\n1.. Předmětem této Smlouvy...,Předmět smlouvy\n\n1.. Předmětem této Smlouvy ...


### Subject extraction

In [12]:
from utils.subject_extraction import *

In [13]:
from utils.subject_context_preprocessing import *

In [14]:
from utils.conllu_preprocessing import *

In [22]:
%%time

transformers = [
    NumeralLinesFilter(too_many_numerals_ratio_threshold=0.5),
    TooShortLinesFilter(too_short_line_threshold=5),
    IrrelevantLinesFilter(keywords=['strana', 'stránka', 'e-mail'], max_line_length=75, lower=True),
    IrrelevantLinesFilter(keywords=['Tel:', 'Fax:', 'IČ:', 'IČO:', 'DIČ:'], max_line_length=75,
                          lower=False),
    IrrelevantLinesRegexFilter(patterns=[r'www', r'[\w\-\.]+\s*@\s*([\w\-]+\.)+[\w\-]{2,4}']),  # email
    IrrelevantLinesRegexFilter(patterns=[r'(\+\d{2,3}){0,1}(\s{0,1}\d{3}){3}']),  # phone
    RegexReplaceTransformer(pattern_to_transform=r',([\s]+[A-Z][a-z ])', result_pattern='.\g<1>'),  # . vs , correction
    RegexReplaceTransformer(pattern_to_transform=r'\n[ \t]*([^\d]]{0,1}[\d]{1,2}[^\d]{0,1})+[ \t]*',  # paragraph numbers
                            result_pattern='\n'),
    BlankLinesFilter(replacement='\n', top_n_frequency=200, top_n_var_threshold=5,
                     full_line_threshold=0.85, min_max_line_length=0),
    ReplaceMarksTransformer(marks_to_transform='„“', result_mark='"'),
    TooLongLinesTransformer(forbidden_delimiters='aábcčdďeéěfghiíjklmnňoópqrřsštťuúůvwxyýzž0123456789',
                            special_delimiters={'-': (r'[\s,\.](-)[\s]+[^(Kč)]', 1)},
                            too_long_line_treshold=200),
    RegexReplaceTransformer(pattern_to_transform=r'([^\n])[ ]*\n', result_pattern='\g<1>.\n'),  # . filling
    RegexReplaceTransformer(pattern_to_transform=r'(([Nn]ázev|[Pp]opis)[^\n,.:"()]{5,})(\s[A-Z][^\n:]{10})',
                            result_pattern='\g<1>:\g<3>'),  # : filling
    ReplaceMarksTransformer(marks_to_transform=[':'], result_mark=':.'),
    ReplaceMarksTransformer(marks_to_transform=['..'], result_mark='.'),
    RegexReplaceTransformer(pattern_to_transform=r'\([^\n()]*\)', result_pattern=''),  # bracket erasing
]
preprocessor = SubjectContextPreprocessor(transformers)
df_contracts['filtered_context'] = df_contracts['subj_context'].apply(lambda data: preprocessor.process(data, True))
# df_contracts

SubjectContextPreprocessor: Elapsed time: 0.0034 seconds
SubjectContextPreprocessor: Elapsed time: 0.0205 seconds
SubjectContextPreprocessor: Elapsed time: 0.0023 seconds
SubjectContextPreprocessor: Elapsed time: 0.0017 seconds
SubjectContextPreprocessor: Elapsed time: 0.0253 seconds
SubjectContextPreprocessor: Elapsed time: 0.0015 seconds
SubjectContextPreprocessor: Elapsed time: 0.0116 seconds
SubjectContextPreprocessor: Elapsed time: 0.0014 seconds
SubjectContextPreprocessor: Elapsed time: 0.0010 seconds
SubjectContextPreprocessor: Elapsed time: 0.0025 seconds
SubjectContextPreprocessor: Elapsed time: 0.0127 seconds
SubjectContextPreprocessor: Elapsed time: 0.0074 seconds
SubjectContextPreprocessor: Elapsed time: 0.0021 seconds
SubjectContextPreprocessor: Elapsed time: 0.0042 seconds
SubjectContextPreprocessor: Elapsed time: 0.0057 seconds
SubjectContextPreprocessor: Elapsed time: 0.0031 seconds
SubjectContextPreprocessor: Elapsed time: 0.0011 seconds
SubjectContextPreprocessor: Ela

SubjectContextPreprocessor: Elapsed time: 0.0048 seconds
SubjectContextPreprocessor: Elapsed time: 0.0035 seconds
SubjectContextPreprocessor: Elapsed time: 0.0014 seconds
SubjectContextPreprocessor: Elapsed time: 0.0030 seconds
SubjectContextPreprocessor: Elapsed time: 0.0038 seconds
SubjectContextPreprocessor: Elapsed time: 0.0028 seconds
SubjectContextPreprocessor: Elapsed time: 0.0040 seconds
SubjectContextPreprocessor: Elapsed time: 0.0008 seconds
SubjectContextPreprocessor: Elapsed time: 0.0007 seconds
SubjectContextPreprocessor: Elapsed time: 0.0021 seconds
SubjectContextPreprocessor: Elapsed time: 0.0027 seconds
SubjectContextPreprocessor: Elapsed time: 0.0031 seconds
SubjectContextPreprocessor: Elapsed time: 0.0037 seconds
SubjectContextPreprocessor: Elapsed time: 0.0022 seconds
SubjectContextPreprocessor: Elapsed time: 0.0022 seconds
SubjectContextPreprocessor: Elapsed time: 0.0158 seconds
SubjectContextPreprocessor: Elapsed time: 0.0228 seconds
SubjectContextPreprocessor: Ela

In [23]:
%%time 
attribute_transformers = [
    QuotedContractNameExtractor(),
    StructuredContractNameExtractor(),
    ItemColonExtractor(),
    StructureItemEnumerationExtractor(),
    CharItemEnumerationExtractor(),
    AttributeExtractor(keep_text=False, keep_attributes=True)
]
attribute_extractor = SubjectContextPreprocessor(transformers = attribute_transformers)
df_contracts['attributes'] = df_contracts['filtered_context'].apply(lambda data: attribute_extractor.process(data))
# df_contracts

Wall time: 835 ms


In [16]:
%%time

annotator = TextAnnotator(pipeline=udp_pipeline)

df_contracts['subj_context_decomposition'] = df_contracts['filtered_context'].apply(lambda data: annotator.process(data))
# df_contracts

KeyError: 'filtered_context'

In [17]:
%%time

from_conllu = UdapiFromConlluTransformer()

df_contracts['subj_document'] = df_contracts['subj_context_decomposition'].apply(lambda data: from_conllu.process(data))
# df_contracts

KeyError: 'subj_context_decomposition'

In [18]:
%%time

conllu_transformers = [
    UdapiWordOccurrencePartSentenceFilter(keywords=['cena', 'hodnota', 'DPH']),
    UdapiWordOccurrencePartSentenceFilter(keywords=['příloha', 'dále', 'jen']),
    NonSubjectPartSentenceFilter(),
    EmptyBundlesFilter(),
]
conllu_preprocessor = ConlluSubjectContextPreprocessor(transformers=conllu_transformers)
df_contracts['filtered_document'] = df_contracts['subj_document'].apply(lambda doc: conllu_preprocessor.process(doc))
# df_contracts

KeyError: 'subj_document'

In [19]:
udapi_tostr = UdapiToStrTransformer()

df_contracts['filtered_document_text'] = df_contracts['filtered_document'].apply(lambda data: udapi_tostr.process(data))
# df_contracts

KeyError: 'filtered_document'

In [ ]:
tagger = AttributeTagger(attr_tag='<ITEM>;<ITEM/>', keep_text=False)

df_contracts['attributes2'] = df_contracts['filtered_document_text'].apply(lambda data: apply_transformation(data, tagger.process))
# df_contracts

In [ ]:
def merge_attributes(row):
    if isinstance(row['attributes'], list):
        return [attrs + '\n' + attrs2 for attrs, attrs2 in zip(row['attributes'], row['attributes2'])]
    return row['attributes'] + '\n' + row['attributes2']

df_contracts['merged_attributes'] = df_contracts.apply(lambda row: merge_attributes(row), axis=1)
# df_contracts

### Measurements

### Playground

In [ ]:
# 4 cena
# structure itemize 14

In [24]:
index = 17
for data in df_contracts.loc[index, 'attributes']:
    print(data)
    print('==========')


<ITEM>Část 4 - Servery<ITEM/>
<ITEM>Část 5 - Disková pole<ITEM/>
<ITEM>Část 6 — Příslušenství<ITEM/>
<ITEM>200000-1 - Počítače<ITEM/>



In [25]:
for data in df_contracts.loc[index, 'filtered_context']:
    print(data)
    print('==========')

Popis předmětu veřejné zakázky.
Předmětem veřejné zakázky zadané v dynamickém nákupním systému  je v 1. části dodávka notebooků.

CPV kódy:.
200000-1 — Počítače.
230000-0 — Zařízení související s počítači.
232000-4 — Periferní vybavení.
30232100-5 - Plottery.

C SYSTEM CZ a.s. s celkovou nabídkovou cenou 11 096 420,- Kč bez DPH.
 (NZM 124 080,- Kč bez DPH. Povodí Odry 99 980,- Kč bez DPH. Povodí Ohře 242 380,-.
Kč bez DPH. Povodí Labe 1 226 250,- Kč bez DPH. Povodí Vltavy 887 200,- Kč bez DPH, SZPI 1 801 200,- Kč bez DPH, SSZPLS 112 080,- Kč bez DPH, ÚKZŮZ 4 221 000,- Kč bez DPH, VŮZV 320 540,- Kč bez DPH, VŮRV 31 020,- Kč bez DPH, SPŮ 1 812 700,- Kč bez DPH, SVS 151 220,- Kč bez DPH, MZe 66 770,- Kč bez DPH).
Identifikační údaje všech dodavatelů a jejich nabídková cena.

Po ] a — p P . |.
| Číslo | Obchodní firma / název Nabídková cena Pořadí |.
| nabídky m | . |.

Azenet s.r.o. k.
1/1. | Plánská 403/5, 301 00 Plzeň a :. 2.

| C SYSTEM CZ a.s. |.
1/2 | O. Ševčíka 840/10, 63600 Brno 11

In [31]:
for data in df_contracts.loc[index, 'subj_context']:
    print(data)
    print('==========')

Popis předmětu veřejné zakázky
Předmětem veřejné zakázky zadané v dynamickém nákupním systému (dále jen „DNS“) je
v 1. části dodávka notebooků.

CPV kódy:

30200000-1 — Počítače

30230000-0 — Zařízení související s počítači

30232000-4 — Periferní vybavení
30232100-5 - Plottery
48820000-2 - Servery

:vw

C SYSTEM CZ a.s. s celkovou nabídkovou cenou 11 096 420,- Kč bez DPH.

(NZM 124 080,- Kč bez DPH, Povodí Odry 99 980,- Kč bez DPH, Povodí Ohře 242 380,-
Kč bez DPH, Povodí Labe 1 226 250,- Kč bez DPH, Povodí Vltavy 887 200,- Kč bez DPH,
SZPI 1 801 200,- Kč bez DPH, SSZPLS 112 080,- Kč bez DPH, ÚKZŮZ 4 221 000,- Kč
bez DPH, VŮZV 320 540,- Kč bez DPH, VŮRV 31 020,- Kč bez DPH, SPŮ 1 812 700,- Kč
bez DPH, SVS 151 220,- Kč bez DPH, MZe 66 770,- Kč bez DPH).

3. Identifikační údaje všech dodavatelů a jejich nabídková cena

 

Po ] a — p P . |
| Číslo | Obchodní firma / název Nabídková cena Pořadí |
| nabídky m | . |

Azenet s.r.o. k
11 204 066,- K
1/1. | Plánská 403/5, 301 00 Plzeň a : 2.
|

In [28]:
text = df_contracts.loc[index, 'text']
print(text)


<FILE id=467>
Národní zemědělské muzeum
	Pověřujcí zadavatel	Národní zemědělské muzeum(NZM)
	IČO	75075741
	Kontaktní osoba	Mgr. Antonín Juriga, antonin.juriga@nzm.cz
	Kategorie	Požadavek (počet kusů)
	PC	12
	PC-klávesnice 1	12
	Monitor kat. 1	16
	Technická specifikace PC	Národní zemědělské muzeum
	Výrobce a modelové označení PC:	[DOPLNÍ UCHAZEČ]	[DOPLNÍ UCHAZEČ]
	Provedení case:	desktop SFF, včetně zdroje s účinností 80 PLUS a vstupním napětí 230V	[DOPLNÍ UCHAZEČ]
		určený pro provoz počítače v horizontální i vertikální poloze	[DOPLNÍ UCHAZEČ]
		jednoduchá modulární konstrukce s možností rychlého otevření skříně a rychlé výměny základních komponent bez použití nástrojů	[DOPLNÍ UCHAZEČ]
		detekce vniknutí do skříně (otevření skříně) s hlášením	[DOPLNÍ UCHAZEČ]
		možnost zabezpečení proti krádeži a s možností uzamčení	[DOPLNÍ UCHAZEČ]
	Rozměry case:	Maximální rozměry: výška 110 mm, šířka 340 mm, hloubka 390 mm	[DOPLNÍ UCHAZEČ]
	Operační systém:	Windows 10 Pro CZ 64 bit	[DOPLNÍ UCHAZEČ]
